In [ ]:
#条件：
#下記のコードで学習してからではないと関数predictが動かないので、一度実行をお願いします。
#学習時間が長いのでもし短縮したい場合、MAX_EPOCH = 5の数字を小さくしてください。
#関数predictの引数は画像のファイル名が入るります。
#この関数は画像に写っているものの推察をしてくれますが、飛行機、車、鳥、猫、鹿、犬、カエル、馬、船、トラックのいずれかしか判断できません。
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import Activation, Flatten, Dense
from keras.optimizers import Adam

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

# 学習データ
train_data_with_teacher_labels = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_data_loader = torch.utils.data.DataLoader(train_data_with_teacher_labels, batch_size=4, shuffle=True, num_workers=2)

# 検証データ
test_data_with_teacher_labels = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_data_loader = torch.utils.data.DataLoader(test_data_with_teacher_labels, batch_size=4, shuffle=False, num_workers=2)

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.layer1 = nn.Linear(16 * 5 * 5, 120)
        self.layer2 = nn.Linear(120, 84)
        self.layer3 = nn.Linear(84, 10)

    def forward(self, input_data):
        input_data = self.pool(F.relu(self.conv1(input_data)))
        input_data = self.pool(F.relu(self.conv2(input_data)))
        input_data = input_data.view(-1, 16 * 5 * 5)
        input_data = F.relu(self.layer1(input_data))
        input_data = F.relu(self.layer2(input_data))
        input_data = self.layer3(input_data)
        return input_data


model = Sequential()

model.add(Conv2D(filters=10, kernel_size=(3, 3), padding='same', input_shape=(32,32,3), activation='relu'))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# エポック数
MAX_EPOCH = 5


# ニューラルネットワークの定義
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.layer1 = nn.Linear(16 * 5 * 5, 120)
        self.layer2 = nn.Linear(120, 84)
        self.layer3 = nn.Linear(84, 10)

    def forward(self, input_data):
        input_data = self.pool(F.relu(self.conv1(input_data)))
        input_data = self.pool(F.relu(self.conv2(input_data)))
        input_data = input_data.view(-1, 16 * 5 * 5)
        input_data = F.relu(self.layer1(input_data))
        input_data = F.relu(self.layer2(input_data))
        input_data = self.layer3(input_data)
        return input_data

# transform定義
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# 学習データ
train_data_with_teacher_labels = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_data_loader = torch.utils.data.DataLoader(train_data_with_teacher_labels, batch_size=4, shuffle=True, num_workers=2)

model = CNN()
# optimizerの設定
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(params=model.parameters(), lr=0.001, momentum=0.9)

# 学習
for epoch in range(MAX_EPOCH):
    total_loss = 0.0
    for i, data in enumerate(train_data_loader, 0):
        # 学習データと教師ラベルデータを取得
        train_data, teacher_labels = data
        # 勾配情報を削除
        optimizer.zero_grad()
        # モデルで予測を計算
        outputs = model(train_data)
        # 微分計算
        loss = criterion(outputs, teacher_labels)
        loss.backward()
        # 勾配を更新
        optimizer.step()
        # 誤差
        total_loss += loss.item()
        # 1000ミニバッチずつ進捗を表示
        if i % 1000 == 999:
            print('学習進捗：[学習回数：%d, ミニバッチ数：%5d] loss: %.3f' % (epoch + 1, i + 1, total_loss / 1000))
            total_loss = 0.0

# モデル保存
torch.save(model, "model.pth")
print("-----学習完了-----")

def predict(img):
  import cv2
  import matplotlib.pyplot as plt
  import torch
  import torchvision
  import torchvision.transforms as transforms
  import torch.nn as nn
  import torch.nn.functional as F
  import torch.optim as optim
  from torchvision import models
  from PIL import Image
  import matplotlib.pyplot as plt

  model = torch.load('model.pth')
  classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

  # 推論モード
  model.eval()

  # 画像をモデルと形式を合わせる
  sample = cv2.imread(img)
  sample_re = cv2.resize(sample, dsize=(32,32))
  plt.imshow(cv2.cvtColor(sample, cv2.COLOR_BGR2RGB))
  sample_t = torchvision.transforms.functional.to_tensor(sample_re)
  x = sample_t.unsqueeze(0)

  # 推論開始
  y_predict = model(x)
  print(y_predict)
  print("推論結果: ", classes[y_predict.argmax()])

In [ ]:
#フォルダにsample.jpgを入れてから実行してください。
predict("sample.jpg")